## 3-step process to reconstruct paleolocations:
1. combine input points into feature collection
2. assign plate ids to features
3. reconstruct paleolocations for features


In [52]:
import sys

import pandas as pd
import numpy as np

import os
import json

import xarray as xr


In [56]:
cwd = os.getcwd()

# load safloor age grid
ds = xr.open_dataset(cwd + '/Muller_etal_2019_Tectonics_v2.0_PresentDay_AgeGrid.nc')


lon = np.arange(-180, 180.1, 0.5).tolist()
lat = np.arange(90, -90.1, -0.5).tolist()

# create array to store 2d fields for each timeslice
validAges = np.zeros((1, len(lat), len(lon)))
print(validAges.shape)

# lon[0] = -179.9
# lon[-1] =  179.9
# lat[0] = ( lat[0] + lat[1] ) / 2.
# lat[-1] = lat[0] * -1.

print(lon)
print(lat)

print(ds)

(1, 361, 721)
[-180.0, -179.5, -179.0, -178.5, -178.0, -177.5, -177.0, -176.5, -176.0, -175.5, -175.0, -174.5, -174.0, -173.5, -173.0, -172.5, -172.0, -171.5, -171.0, -170.5, -170.0, -169.5, -169.0, -168.5, -168.0, -167.5, -167.0, -166.5, -166.0, -165.5, -165.0, -164.5, -164.0, -163.5, -163.0, -162.5, -162.0, -161.5, -161.0, -160.5, -160.0, -159.5, -159.0, -158.5, -158.0, -157.5, -157.0, -156.5, -156.0, -155.5, -155.0, -154.5, -154.0, -153.5, -153.0, -152.5, -152.0, -151.5, -151.0, -150.5, -150.0, -149.5, -149.0, -148.5, -148.0, -147.5, -147.0, -146.5, -146.0, -145.5, -145.0, -144.5, -144.0, -143.5, -143.0, -142.5, -142.0, -141.5, -141.0, -140.5, -140.0, -139.5, -139.0, -138.5, -138.0, -137.5, -137.0, -136.5, -136.0, -135.5, -135.0, -134.5, -134.0, -133.5, -133.0, -132.5, -132.0, -131.5, -131.0, -130.5, -130.0, -129.5, -129.0, -128.5, -128.0, -127.5, -127.0, -126.5, -126.0, -125.5, -125.0, -124.5, -124.0, -123.5, -123.0, -122.5, -122.0, -121.5, -121.0, -120.5, -120.0, -119.5, -119.0, -

In [67]:
# create rotationTextures (paleolat and paleolon for modern lat/lon) for each time period

#origLats = ds['z'].coords['lat']


#### loop over all gridpoints
for jj in range(len(lat)):
    print(jj)

    for ii in range(len(lon)):
        # index by value        
        age = ds['z'].sel(lon=lon[ii], lat=lat[jj], method="nearest")
        if (np.isnan(age)):
            validAges[0, jj, ii] = 550.
        else:
            validAges[0, jj, ii] = age


                

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [69]:
# write textures to disk

# function to normalize data to range 0-1
def norm(arr, norm_min, norm_max):
    arr_min =  norm_min
    arr_max =  norm_max
    return (arr - norm_min) / (norm_max - norm_min)
    

outputData  = np.zeros( ( len(lat), len(lon) , 1) )

outputData[:,:,0] = norm(validAges[0,:,:], 0., 550.) * 255 
# cap data to chosen dynamic range
outputData[outputData > 255] = 255
outputData[outputData < 0] = 0
outputData = outputData.astype(np.uint8)
# outputData[outputData > 65535] = 65535
# outputData[outputData < 0] = 0
# outputData = outputData.astype(np.uint16)
import png

with open(cwd + '/climatearchive/ageTextures/Mueller2019.validAges.png', 'wb') as f:
    writer = png.Writer(width=outputData.shape[1], height=outputData.shape[0], bitdepth=8, greyscale=True)
    # Convert outputData_3D_ym to the Python list of lists expected by the png writer.
    outputData_ym2list = outputData.reshape(-1, outputData.shape[1]*outputData.shape[2]).tolist()
    writer.write(f, outputData_ym2list)